# Rewrite `_base` function to fit more jobs.
Also, start learning to simplify what is needed.

In [2]:
# Import relevant modules
import os           # interface with the underlying OS
import numpy as np  # linear algebra
import pandas as pd # dataframe
import re           # regular expression

In [3]:
# Define path to look at
path = 'CityofLA/Job Bulletins/'

# Get a list of all txt files in this path
all_txt_files = os.listdir(path)
all_txt_files.sort() # WARNING: this mutates the list

In [4]:
# Do some checks here
print(len(all_txt_files))      # length should be 683 as manually verified
print(len(set(all_txt_files))) # see if each file is unique. hopefully 683 as well!
print('SYSTEMS ANALYST 1596 102717.txt' in all_txt_files) # should be True

683
683
True


In [5]:
# Find `SYSTEMS ANALYST 1596 102717.txt` (which exists).
print(all_txt_files.index('SYSTEMS ANALYST 1596 102717.txt'))

609


## Import file as text
`sa` stands for systems analyst

In [6]:
sa_path = path + all_txt_files[609]
sa = open(sa_path, 'rt').read()
sa # display

'SYSTEMS ANALYST\n\nClass Code: 1596\nOpen Date: 10-27-17\n(Exam Open to All, including Current City Employees)\n\nANNUAL SALARY\n\n$68,611 to $100,307\nThe salary in the Department of Water and Power is $70,908 to $88,092 and $83,770 to $104,065.\n\nNOTES:\n\n1. Candidates from the eligible list are normally appointed to vacancies in the lower pay grade positions.  \n2. Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.\n3. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.\n\n\nDUTIES\n\nA Systems Analyst analyzes procedures, methods and operations of computer-based information systems; designs, implements, and recommends information systems to improve the efficiency and economy of City operations; performs cost benefit and feasibility analyses re

## First, do individual parts 

<font color='red'> Need to use regex for a better information location. </font>

In [7]:
# Locate the information
temp = sa[sa.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):sa.find('PROCESS NOTES')]
temp

'REQUIREMENTS/MINIMUM QUALIFICATIONS\n\n1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or\n2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:\na. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or\nb. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or\nc. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or\n3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and\na. Satisfactory completion of four cou

In [8]:
# Fix the above problem
digits_as_str = [sentence[-1] for sentence in temp.split('.') if sentence[-1].isdigit()]
digits_as_str

['1', '2', '3']

In [9]:
# Separate requirements one from the other.
## Initialization
line_item = [] # borrow from accounting term. means items in a list
## Loop through idx of digits_as_str to get the text from one digit to the other
for idx in range(len(digits_as_str)-1):
    start = digits_as_str[idx]  ; pos1 = temp.index(start)
    end   = digits_as_str[idx+1]; pos2 = temp.index(end)
    item  = temp[pos1:pos2]     ; line_item.append(item)
## Put the last part into the list of all possible requirements
final_dig    = digits_as_str[-1]; final_pos = temp.index(final_dig)
final_item = temp[final_pos:] ; line_item.append(final_item)
## Display results
line_item

['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or\n',
 '2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:\na. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or\nb. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or\nc. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or\n',
 '3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and\na. Satisfactory completion of four courses, of at least three seme

In [10]:
# Split further by newline characters
line_item = [[line.split(';') for line in item.split('\n') if len(line) > 1] for item in line_item]
line_item

[[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
   ' or']],
 [['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:'],
  ['a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases',
   ' or'],
  ['b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system',
   ' or'],
  ['c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation',
   ' or']],
 [['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
   ' and'],
  ['a. Sat

In [11]:
# Remove one layer of nested list.
# Source: https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
line_item = [[item for sublist in l for item in sublist] for l in line_item]
line_item

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
  ' or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases',
  ' or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system',
  ' or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation',
  ' or'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
  ' and',
  'a. Satisfactory completi

In [12]:
# Simplify line_item by throwing away anything after the punctuation of the main sentence,
# if the main sentence endswith a punctuation. This is to handle `CARPENTER 3344 011218.txt`
for aList in line_item:
    for item in list(aList):
        if len(item) > 5:
            if item.split()[0][1] != '.':
                aList.remove(item)
            
line_item

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
  ' or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases',
  ' or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system',
  ' or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation',
  ' or'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
  ' and',
  'a. Satisfactory completi

In [135]:
for aList in line_item:
    for idx in range(len(aList)):
        aList[idx]=aList[idx].split(';')

In [128]:
[u[0] for u in line_item]

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
  ' or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
  ' and']]

In [138]:
line_item

[[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
   ' or']],
 [['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:'],
  ['a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases',
   ' or'],
  ['b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system',
   ' or'],
  ['c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation',
   ' or']],
 [['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
   ' and'],
  ['a. Sat

In [146]:
k = []
for l in line_item:
    k.append([item for sublist in l for item in sublist])
    
k

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
  ' or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases',
  ' or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system',
  ' or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation',
  ' or'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
  ' and',
  'a. Satisfactory completi

In [145]:
l

['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
 ' and',
 'a. Satisfactory completion of four courses, of at least three semester or four quarter units each, in Information Systems, Systems Analysis, or a closely related degree program, professional designation, or certificate program from an accredited college or university.',
 'b. At least three of the courses must be from the core courses required in the program, and one course may be from either the required core courses or the prescribed elective courses of the program.  A course in systems analysis and design is especially desired, but not required. ']

In [139]:
l = [[1], [2, 3]]
[item for sublist in l for item in sublist]

[1, 2, 3]

In [122]:
line_item

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and',
  'a. Satisfactory completion of four courses, of at

In [77]:
line_item

['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or\n',
 '2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:\na. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or\nb. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or\nc. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or\n',
 '3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and\na. Satisfactory completion of four courses, of at least three seme

In [94]:
u = [[line.split(';') for line in item.split('\n') if len(line) > 1] for item in line_item]

In [85]:
[item.split('\n') for item in line_item[1].split(';')]

[['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases'],
 [' or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system'],
 [' or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation'],
 [' or', '']]

In [89]:
[[line for line in item.split(';')] for item in line_item]

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
  ' or\n'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:\na. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases',
  ' or\nb. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system',
  ' or\nc. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation',
  ' or\n'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
  ' and\na. Satisfactory completion of four c

In [102]:
# Split further by newline characters
line_item = [[line for line in item.split('\n') if len(line) > 1] for item in line_item]
line_item

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and',
  'a. Satisfactory completion of four courses, of at

In [100]:
line_item

['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or\n',
 '2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:\na. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or\nb. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or\nc. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or\n',
 '3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and\na. Satisfactory completion of four courses, of at least three seme

In [99]:
for aList in line_item:
    print(aList)

1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or

2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:
a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or
b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or
c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or

3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and
a. Satisfactory completion of four courses, of at least three semester or four qua

In [93]:
line_item[1].split('\n')

['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
 'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or',
 'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or',
 'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or',
 '']

In [72]:
# Simplify line_item by throwing away anything after the punctuation of the main sentence,
# if the main sentence endswith a punctuation. This is to handle `CARPENTER 3344 011218.txt`
for aList in line_item:
    for item in list(aList):
        if item.split()[0][1] != '.':
            aList.remove(item)
            
line_item

[['1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems; or'],
 ['2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  'a. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases; or',
  'b. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system; or',
  'c. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation; or'],
 ['3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and',
  'a. Satisfactory completion of four courses, of at

In [73]:
# Why is it so hardddd????

In [75]:
temp.split('; or')

['REQUIREMENTS/MINIMUM QUALIFICATIONS\n\n1. Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
 '\n2. Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:\na. the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases',
 '\nb. performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system',
 '\nc. performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation',
 '\n3. Two years of full-time paid experience as a Systems Aide with the City of Los Angeles; and\na. Satisfactory completion of fou

In [13]:
requirement_list = []
for aList in line_item:
    # First item in the list aList may be a requirement
    # requirement_id is the first element in the string
    # requirement_txt is anything after, for example, 1., 2., 3., etc. (2 characters, that's why 2:)
    main_information = aList[0]
    requirement_id   = main_information[0]; requirement_txt = main_information[2:]
    # Step 1
    if len(aList) == 1:
        ## Step 1a: Handle case "1. and 2.", "1. or 2."
        if main_information.endswith('and'):
            ### Find next item's requirement id and the text associated with it
            next_list_idx   = line_item.index(aList)+1; next_list = line_item[next_list_idx]; next_req = next_list[0]
            next_digit      = next_req[0]             ; next_req_txt = next_req[2:]
            ### Form a nested requirement and put into requirement list
            nested_reqmnt   = (requirement_id, requirement_txt, next_digit, next_req_txt)
            requirement_list.append(nested_reqmnt)
        ## Step 1b: Handle case "1. or 2."
        else:
            requirement = (requirement_id, requirement_txt, 'A')
            requirement_list.append(requirement)

    # Step 2
    elif len(aList) > 1:
        # Step 2a: Handle case "1. a. or b."
        if all([sub_item.endswith('or') for sub_item in aList[1:-1]]):
            all_sub_requirements = aList[1:]
            for sub_requirement in all_sub_requirements:
                requirement_subset_id  = sub_requirement[0].upper(); requirement_subset_txt = sub_requirement[2:]
                nested_reqmnt = (requirement_id, requirement_txt, requirement_subset_id, requirement_subset_txt)
                requirement_list.append(nested_reqmnt)
        # Step 2b: Handle case "3. a. and b."
        else:
            nested_reqmnt = (requirement_id, requirement_txt)
            for possibly_a_sub_requirement in aList[1:]:
                ## Query the previous item to see if it ends with 'and'. For example, 3. ... and
                previous_item_idx = aList.index(possibly_a_sub_requirement)-1
                previous_item_txt = aList[previous_item_idx]
                if previous_item_txt.endswith('and'):
                    sub_requirement_id  = possibly_a_sub_requirement[0].upper()
                    sub_requirement_txt = possibly_a_sub_requirement[2:]
                    nested_reqmnt      += (sub_requirement_id, None, sub_requirement_txt)
                else:
                    miscellaneous_txt   = possibly_a_sub_requirement[2:]
                    nested_reqmnt      += (miscellaneous_txt, )
            requirement_list.append(nested_reqmnt)

# Normalize each element in requirement list to length 6
requirement_list = [tup+(None, )*(6-len(tup)) for tup in requirement_list]

# Display result
requirement_list

[('1',
  ' Graduation from an accredited four-year college or university with a major in Computer Science, Information Systems, or Geographical Information Systems',
  ' ',
  'r',
  None,
  None),
 ('2',
  ' Graduation from an accredited four-year college or university and two years of full-time paid experience in a class at the level of Management Assistant which provides experience in:',
  ' the development, analysis, implementation or major modification of new or existing computer-based information systems or relational databases',
  'r',
  ' performing cost benefit, feasibility and requirements analysis for a large-scale computer-based information system',
  'r',
  ' performing system implementation and support activities including software and hardware acquisition, installation, modifications to system configuration, system and application upgrade installation',
  'r'),
 ('3',
  ' Two years of full-time paid experience as a Systems Aide with the City of Los Angeles',
  'nd',
  'A'

In [11]:
pd.DataFrame(requirement_list)

,0,1,2,3,4,5
0,1,Graduation from an accredited four-year colle...,A,None,None,None
1,2,Graduation from an accredited four-year colle...,A,"the development, analysis, implementation or ...",None,None
2,2,Graduation from an accredited four-year colle...,B,"performing cost benefit, feasibility and requ...",None,None
3,2,Graduation from an accredited four-year colle...,C,performing system implementation and support ...,None,None
4,3,Two years of full-time paid experience as a S...,A,None,"Satisfactory completion of four courses, of a...",At least three of the courses must be from th...


In [12]:
# Define a helper function
def _base(job, use_df=True):
    '''
    _BASE attempts to parse the structure in REQUIREMENTS/MINIMUM QUALIFICATIONS. 
    It takes in a job and returns a dataframe of information, which supports other functions.
    
    It is intended to use inside other functions (hence started with an underscore), not for stand-alone use.
    '''
    # Each hash # below is the code in one of the cell above. For output/logic, see above cells.
    
    # From REQUIREMENTS/MINIMUM QUALIFICATIONS to PROCESS NOTES is where the information located
    temp = job[job.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):job.find('PROCESS NOTES')]
    
    # Get the digits that mark one part from the other
    digits_as_str = [sentence[0] for sentence in temp.split() if sentence[0].isdigit()]
    
    # Separate requirements one from the other.
    line_item = [] # borrow from accounting term. means items in a list
    ## Loop through idx of digits_as_str to get the text from one digit to the other
    for idx in range(len(digits_as_str)-1):
        start = digits_as_str[idx]  ; pos1 = temp.index(start)
        end   = digits_as_str[idx+1]; pos2 = temp.index(end)
        item  = temp[pos1:pos2]     ; line_item.append(item)
    ## Put the last part into the list of all possible requirements
    final_dig    = digits_as_str[-1]; final_pos = temp.index(final_dig)
    final_item = temp[final_pos:] ; line_item.append(final_item)

    # Split further by newline characters
    line_item = [[line for line in item.split('\n') if len(line) > 1] for item in line_item]
    
    # Implementation of the algorithm
    requirement_list = []
    for aList in line_item:
        ## First item in the list aList may be a requirement
        ## requirement_id is the first element in the string
        ## requirement_txt is anything after, for example, 1., 2., 3., etc. (2 characters, that's why 2:)
        main_information = aList[0]
        requirement_id   = main_information[0]; requirement_txt = main_information[2:]
        ## Step 1
        if len(aList) == 1:
            ### Step 1a: Handle case "1. and 2.", "1. or 2."
            if main_information.endswith('and'):
                #### Find next item's requirement id and the text associated with it
                next_list_idx   = line_item.index(aList)+1; next_list = line_item[next_list_idx]; next_req = next_list[0]
                next_digit      = next_req[0]             ; next_req_txt = next_req[2:]
                #### Form a nested requirement and put into requirement list
                nested_reqmnt   = (requirement_id, requirement_txt, next_digit, next_req_txt)
                requirement_list.append(nested_reqmnt)
            ### Step 1b: Handle case "1. or 2."
            else:
                requirement = (requirement_id, requirement_txt, 'A')
                requirement_list.append(requirement)
        ## Step 2
        elif len(aList) > 1:
            ## Step 2a: Handle case "1. a. or b."
            if all([sub_item.endswith('or') for sub_item in aList[1:-1]]):
                all_sub_requirements = aList[1:]
                for sub_requirement in all_sub_requirements:
                    requirement_subset_id  = sub_requirement[0].upper(); requirement_subset_txt = sub_requirement[2:]
                    nested_reqmnt = (requirement_id, requirement_txt, requirement_subset_id, requirement_subset_txt)
                    requirement_list.append(nested_reqmnt)
            ## Step 2b: Handle case "3. a. and b."
            else:
                nested_reqmnt = (requirement_id, requirement_txt)
                for possibly_a_sub_requirement in aList[1:]:
                    ### Query the previous item to see if it ends with 'and'. For example, 3. ... and
                    previous_item_idx = aList.index(possibly_a_sub_requirement)-1
                    previous_item_txt = aList[previous_item_idx]
                    if previous_item_txt.endswith('and'):
                        sub_requirement_id  = possibly_a_sub_requirement[0].upper()
                        sub_requirement_txt = possibly_a_sub_requirement[2:]
                        nested_reqmnt      += (sub_requirement_id, None, sub_requirement_txt)
                    else:
                        miscellaneous_txt   = possibly_a_sub_requirement[2:]
                        nested_reqmnt      += (miscellaneous_txt, )
                requirement_list.append(nested_reqmnt)
    ## Normalize each element in requirement list to length 6
    requirement_list = [tup+(None, )*(6-len(tup)) for tup in requirement_list]

    # Returns
    df = pd.DataFrame(data=requirement_list, columns=['REQUIREMENT_SET_ID',
                                                      'MINE_1',
                                                      'REQUIREMENT_SUBSET_ID',
                                                      'EXP_JOB_CLASS_FUNCTION',
                                                      'MINE_2',
                                                      'MISC'])
    return df

# Test
print(_base(job=sa).shape)
_base(job=sa)

(5, 6)


,REQUIREMENT_SET_ID,MINE_1,REQUIREMENT_SUBSET_ID,EXP_JOB_CLASS_FUNCTION,MINE_2,MISC
0,1,Graduation from an accredited four-year colle...,A,None,None,None
1,2,Graduation from an accredited four-year colle...,A,"the development, analysis, implementation or ...",None,None
2,2,Graduation from an accredited four-year colle...,B,"performing cost benefit, feasibility and requ...",None,None
3,2,Graduation from an accredited four-year colle...,C,performing system implementation and support ...,None,None
4,3,Two years of full-time paid experience as a S...,A,None,"Satisfactory completion of four courses, of a...",At least three of the courses must be from th...


## Now, let's test it on some other jobs
* `CARPENTER 3344 011218.txt`
* `TREE SURGEON 3114 081916 REV 080818.txt`

CARPENTER 3344 011218.txt

In [14]:
# Find ``CARPENTER 3344 011218.txt`
print(all_txt_files.index('CARPENTER 3344 011218.txt'))

94


In [16]:
# Import file as text
carpenter_path = path + all_txt_files[94]
carpenter = open(carpenter_path, 'rt').read()
carpenter

'CARPENTER\n\nClass Code: 3344\nOpen Date: 01-12-18\n\t                                 (Exam Open to All, including Current City Employees)\n\nANNUAL SALARY\n\n$86,312 (flat-rated)\nThe salary in the Department of Water and Power is $93,542 (flat-rated)\n\nNOTES:\n\n1. For information regarding reciprocity between the City of Los Angeles departments and LADWP, go to: http://per.lacity.org/Reciprocity_CityDepts_and_DWP.pdf.\n2. Annual salary is at the start of the pay range. The current salary range is subject to change. Please confirm the starting salary with the hiring department before accepting a job offer.\n\nDUTIES\n\nA Carpenter does skilled rough and finish carpentry in building and repairing wood structures and articles; work may include forming for the placement of concrete.\n\nREQUIREMENTS/MINIMUM QUALIFICATIONS\n\n1. Completion of an apprenticeship as a carpenter or cabinetmaker; or\n2. Six years of full-time paid experience performing carpenter or cabinetmaker work.\n\nCom

In [17]:
# Test
_base(job=carpenter)

# Oops!

IndexError: list index out of range

In [19]:
# Locate the information
temp = carpenter[carpenter.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):carpenter.find('PROCESS NOTES')]
temp

'REQUIREMENTS/MINIMUM QUALIFICATIONS\n\n1. Completion of an apprenticeship as a carpenter or cabinetmaker; or\n2. Six years of full-time paid experience performing carpenter or cabinetmaker work.\n\nCompletion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork from an accredited institution may be substituted for each year of experience lacking up to a maximum of two years.\n\n'

In [25]:
# Get the digits that mark one part from the other
digits_as_str = [sentence[0] for sentence in temp.split() if sentence[0].isdigit()]
digits_as_str

['1', '2', '1']

In [47]:
# Fix the above problem
digits_as_str = [sentence[-1] for sentence in temp.split('.') if sentence[-1].isdigit()]
digits_as_str

['1', '2']

In [48]:
# Separate requirements one from the other.
## Initialization
line_item = [] # borrow from accounting term. means items in a list
## Loop through idx of digits_as_str to get the text from one digit to the other
for idx in range(len(digits_as_str)-1):
    start = digits_as_str[idx]  ; pos1 = temp.index(start)
    end   = digits_as_str[idx+1]; pos2 = temp.index(end)
    item  = temp[pos1:pos2]     ; line_item.append(item)
## Put the last part into the list of all possible requirements
final_dig    = digits_as_str[-1]; final_pos = temp.index(final_dig)
final_item = temp[final_pos:] ; line_item.append(final_item)
## Display results
line_item

['1. Completion of an apprenticeship as a carpenter or cabinetmaker; or\n',
 '2. Six years of full-time paid experience performing carpenter or cabinetmaker work.\n\nCompletion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork from an accredited institution may be substituted for each year of experience lacking up to a maximum of two years.\n\n']

In [49]:
# Split further by newline characters
line_item = [[line for line in item.split('\n') if len(line) > 1] for item in line_item]
line_item

[['1. Completion of an apprenticeship as a carpenter or cabinetmaker; or'],
 ['2. Six years of full-time paid experience performing carpenter or cabinetmaker work.',
  'Completion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork from an accredited institution may be substituted for each year of experience lacking up to a maximum of two years.']]

In [50]:
requirement_list = []
for aList in line_item:
    # First item in the list aList may be a requirement
    # requirement_id is the first element in the string
    # requirement_txt is anything after, for example, 1., 2., 3., etc. (2 characters, that's why 2:)
    main_information = aList[0]
    requirement_id   = main_information[0]; requirement_txt = main_information[2:]
    # Step 1
    if len(aList) == 1:
        ## Step 1a: Handle case "1. and 2.", "1. or 2."
        if main_information.endswith('and'):
            ### Find next item's requirement id and the text associated with it
            next_list_idx   = line_item.index(aList)+1; next_list = line_item[next_list_idx]; next_req = next_list[0]
            next_digit      = next_req[0]             ; next_req_txt = next_req[2:]
            ### Form a nested requirement and put into requirement list
            nested_reqmnt   = (requirement_id, requirement_txt, next_digit, next_req_txt)
            requirement_list.append(nested_reqmnt)
        ## Step 1b: Handle case "1. or 2."
        else:
            requirement = (requirement_id, requirement_txt, 'A')
            requirement_list.append(requirement)

    # Step 2
    elif len(aList) > 1:
        # Step 2a: Handle case "1. a. or b."
        if all([sub_item.endswith('or') for sub_item in aList[1:-1]]):
            all_sub_requirements = aList[1:]
            for sub_requirement in all_sub_requirements:
                requirement_subset_id  = sub_requirement[0].upper(); requirement_subset_txt = sub_requirement[2:]
                nested_reqmnt = (requirement_id, requirement_txt, requirement_subset_id, requirement_subset_txt)
                requirement_list.append(nested_reqmnt)
        # Step 2b: Handle case "3. a. and b."
        else:
            nested_reqmnt = (requirement_id, requirement_txt)
            for possibly_a_sub_requirement in aList[1:]:
                ## Query the previous item to see if it ends with 'and'. For example, 3. ... and
                previous_item_idx = aList.index(possibly_a_sub_requirement)-1
                previous_item_txt = aList[previous_item_idx]
                if previous_item_txt.endswith('and'):
                    sub_requirement_id  = possibly_a_sub_requirement[0].upper()
                    sub_requirement_txt = possibly_a_sub_requirement[2:]
                    nested_reqmnt      += (sub_requirement_id, None, sub_requirement_txt)
                else:
                    miscellaneous_txt   = possibly_a_sub_requirement[2:]
                    nested_reqmnt      += (miscellaneous_txt, )
            requirement_list.append(nested_reqmnt)

# Normalize each element in requirement list to length 6
requirement_list = [tup+(None, )*(6-len(tup)) for tup in requirement_list]

# Display result
requirement_list

[('1',
  ' Completion of an apprenticeship as a carpenter or cabinetmaker; or',
  'A',
  None,
  None,
  None),
 ('2',
  ' Six years of full-time paid experience performing carpenter or cabinetmaker work.',
  'C',
  'mpletion of a minimum 18-month or longer program in carpentry or cabinetmaking and millwork from an accredited institution may be substituted for each year of experience lacking up to a maximum of two years.',
  None,
  None)]

In [52]:
# Re-define a helper function. In particular, fix the part at digits_as_str
def _base(job, use_df=True):
    '''
    _BASE attempts to parse the structure in REQUIREMENTS/MINIMUM QUALIFICATIONS. 
    It takes in a job and returns a dataframe of information, which supports other functions.
    
    It is intended to use inside other functions (hence started with an underscore), not for stand-alone use.
    '''
    # Each hash # below is the code in one of the cell above. For output/logic, see above cells.
    
    # From REQUIREMENTS/MINIMUM QUALIFICATIONS to PROCESS NOTES is where the information located
    temp = job[job.find('REQUIREMENTS/MINIMUM QUALIFICATIONS'):job.find('PROCESS NOTES')]
    
    # Get the digits that mark one part from the other
    digits_as_str = [sentence[-1] for sentence in temp.split('.') if sentence[-1].isdigit()]
    
    # Separate requirements one from the other.
    line_item = [] # borrow from accounting term. means items in a list
    ## Loop through idx of digits_as_str to get the text from one digit to the other
    for idx in range(len(digits_as_str)-1):
        start = digits_as_str[idx]  ; pos1 = temp.index(start)
        end   = digits_as_str[idx+1]; pos2 = temp.index(end)
        item  = temp[pos1:pos2]     ; line_item.append(item)
    ## Put the last part into the list of all possible requirements
    final_dig    = digits_as_str[-1]; final_pos = temp.index(final_dig)
    final_item = temp[final_pos:] ; line_item.append(final_item)

    # Split further by newline characters
    line_item = [[line for line in item.split('\n') if len(line) > 1] for item in line_item]
    
    # Implementation of the algorithm
    requirement_list = []
    for aList in line_item:
        ## First item in the list aList may be a requirement
        ## requirement_id is the first element in the string
        ## requirement_txt is anything after, for example, 1., 2., 3., etc. (2 characters, that's why 2:)
        main_information = aList[0]
        requirement_id   = main_information[0]; requirement_txt = main_information[2:]
        ## Step 1
        if len(aList) == 1:
            ### Step 1a: Handle case "1. and 2.", "1. or 2."
            if main_information.endswith('and'):
                #### Find next item's requirement id and the text associated with it
                next_list_idx = line_item.index(aList)+1; next_list = line_item[next_list_idx]; next_req=next_list[0]
                next_digit    = next_req[0]             ; next_req_txt = next_req[2:]
                #### Form a nested requirement and put into requirement list
                nested_reqmnt   = (requirement_id, requirement_txt, next_digit, next_req_txt)
                requirement_list.append(nested_reqmnt)
            ### Step 1b: Handle case "1. or 2."
            else:
                requirement = (requirement_id, requirement_txt, 'A')
                requirement_list.append(requirement)
        ## Step 2
        elif len(aList) > 1:
            ## Step 2a: Handle case "1. a. or b."
            if all([sub_item.endswith('or') for sub_item in aList[1:-1]]):
                all_sub_requirements = aList[1:]
                for sub_requirement in all_sub_requirements:
                    requirement_subset_id  = sub_requirement[0].upper(); requirement_subset_txt = sub_requirement[2:]
                    nested_reqmnt = (requirement_id, requirement_txt, requirement_subset_id, requirement_subset_txt)
                    requirement_list.append(nested_reqmnt)
            ## Step 2b: Handle case "3. a. and b."
            else:
                nested_reqmnt = (requirement_id, requirement_txt)
                for possibly_a_sub_requirement in aList[1:]:
                    ### Query the previous item to see if it ends with 'and'. For example, 3. ... and
                    previous_item_idx = aList.index(possibly_a_sub_requirement)-1
                    previous_item_txt = aList[previous_item_idx]
                    if previous_item_txt.endswith('and'):
                        sub_requirement_id  = possibly_a_sub_requirement[0].upper()
                        sub_requirement_txt = possibly_a_sub_requirement[2:]
                        nested_reqmnt      += (sub_requirement_id, None, sub_requirement_txt)
                    else:
                        miscellaneous_txt   = possibly_a_sub_requirement[2:]
                        nested_reqmnt      += (miscellaneous_txt, )
                requirement_list.append(nested_reqmnt)
    ## Normalize each element in requirement list to length 6
    requirement_list = [tup+(None, )*(6-len(tup)) for tup in requirement_list]

    # Returns
    df = pd.DataFrame(data=requirement_list, columns=['REQUIREMENT_SET_ID',
                                                      'MINE_1',
                                                      'REQUIREMENT_SUBSET_ID',
                                                      'EXP_JOB_CLASS_FUNCTION',
                                                      'MINE_2',
                                                      'MISC'])
    return df

# Test
print(_base(job=carpenter).shape)
_base(job=carpenter)

(2, 6)


,REQUIREMENT_SET_ID,MINE_1,REQUIREMENT_SUBSET_ID,EXP_JOB_CLASS_FUNCTION,MINE_2,MISC
0,1,Completion of an apprenticeship as a carpente...,A,None,None,None
1,2,Six years of full-time paid experience perfor...,C,mpletion of a minimum 18-month or longer progr...,None,None


<font size=8, color='red'> Quite messed up. Requires a new approach! </font>
See Notebook `Object1_e`